# Home Credit Default Risk - Team 3 (Kahsai, Nichols, Pellerito)
This notebook handles all of our data cleansing, feature engineering and table merges. These results are passed via pickle into the model notebook.

https://www.kaggle.com/cloycebox/default-risk-week-6-models/edit

### Import packages
The Switch variable (in the first line of code) tells whether to create the training (0) or test (1) data sets. FNs array contains the names of the pickle files that we will save our outputs to.

I considered including the test / validation split and pipeline in this notebook as well, so that the outputs of this file could be X_train, X_valid, X_test, y_train and y_valid (there is no such thing as y_test of course.) Ultimately I decided against this because the most time-consuming part of the whole process is uploading the test and train pickle files into the model notebook - it's >1GB of data and can take 10-15 minutes to upload. Running it through the pipeline and creating dummy columns would only make the data set larger and file transfer slower. So the first steps of the model notebook are reading the pickle files from this notebook and then applying train/valid split and pipeline to them.

In [ ]:
Switch = 1                                                       # 0 creates training data set, 1 creates test data set
DFs = ['application_train.csv', 'application_test.csv']          # which data frame to load, based on switch
FNs = ['train1205.pkl', 'test1205.pkl']                          # what to name pickle output, based on switch

import numpy as np
import pandas as pd
import joblib   # save and load ML models
import gc       # garbage collection
import os 
import pickle

# clear out any old junk
gc.collect()

# define the directory where our stuff lives
MainDir = "../input/../input/home-credit-default-risk"
print(os.listdir(MainDir))

# Building the training / test data set
Strategy:
* merge bureau_balance into bureau using SK_ID_BUREAU key, then merge bureau into train/test using SK_ID_CURR key
* all other tables will be merged directly into train/test using SK_ID_CURR key (not going to use SK_ID_PREV)

I also tried two-step merging, e.g. merging installment_payments into previous_application on SK_ID_PREV, and then merging previous_application into the main data set on SK_ID_CURR. I did not find any appreciable difference between doing it that way and directly merging installment_payments into main on SK_ID_CURR, so I decided to do it the direct way for the final data sets.

Final output tables will be saved to train.pkl and test.pkl

### bureau_balance.csv
load bureau_balance.csv, make two pivot tables (one for overall, one for last 12 months only) - all we can do with them is merge them into bureau using SK_ID_BUREAU because we are not given SK_ID_CURR in this table.

In [ ]:
bureau_balance = pd.read_csv(f'{MainDir}/bureau_balance.csv')                                           # read the table
print(bureau_balance.shape, "- shape of bureau_balance table")
bb_status = pd.crosstab(bureau_balance.SK_ID_BUREAU, bureau_balance.STATUS, margins = True)             # create pivot table
bb_status.columns = ['BB_'+column for column in bb_status.columns]
print(bb_status.shape, "- shape of bb_status table")
bureau_balance_12 = bureau_balance[bureau_balance.MONTHS_BALANCE > -24]                                 # filter - last 12 months only
bb_status_12 = pd.crosstab(bureau_balance_12.SK_ID_BUREAU, bureau_balance_12.STATUS, margins = True)    # pivot table on filtered data
bb_status_12.columns = ['BB12_'+column for column in bb_status_12.columns]
print(bb_status_12.shape, "- shape of bb_status_12 table")

del bureau_balance            # we are keeping bb_status and bb_status_12 - we can drop the original table and the filtered version
del bureau_balance_12
gc.collect()

### bureau.csv
we will create bureau_num, bureau_cat and bureau_count - these will later be merged into train and into test

In [ ]:
bureau = pd.read_csv(f'{MainDir}/bureau.csv')
print(bureau.shape, "- shape of bureau table")

bureau = bureau.merge(bb_status, left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')               # merge the tables
bureau = bureau.merge(bb_status_12, left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')            # merge the tables
bureau = bureau.drop(['SK_ID_BUREAU'], axis = 1)                                                    # no longer need this key
print(bureau.shape, "- shape of bureau table after merging in bb_status tables")                    # should be 652,144 x 34
bureau.columns = ['BU_'+column if column !='SK_ID_CURR' else column for column in bureau.columns]   # things that start with BU_ came from bureau

# ratio features
bureau['OD_ratio'] = bureau['BU_AMT_CREDIT_SUM_OVERDUE'] / bureau['BU_AMT_CREDIT_SUM_DEBT']   # proportion of debt that is overdue
bureau['Credit_ratio'] = bureau['BU_AMT_CREDIT_SUM'] / bureau['BU_AMT_CREDIT_SUM_LIMIT']      # proportion of credit line used
bureau['Debt_ratio'] = bureau['BU_AMT_CREDIT_SUM_DEBT'] / bureau['BU_AMT_CREDIT_SUM']         # debt percentage
bureau['REMAIN_CRED'] = bureau['BU_AMT_CREDIT_SUM'] - bureau['BU_AMT_CREDIT_SUM_DEBT'] - bureau['BU_AMT_CREDIT_SUM_LIMIT']
bureau['AC_RATIO'] = bureau['BU_AMT_ANNUITY'] / bureau['BU_AMT_CREDIT_SUM']

# numeric features for bureau
bureau_num = bureau.groupby(by=['SK_ID_CURR']).mean().reset_index()                                 # group the numeric features by SK_ID_CURR
print(bureau_num.shape, "- shape of numeric bureau features (incl index)")                          # should be 132,250 x 34

# categorical feagures for bureau
bureau_cat = pd.get_dummies(bureau.select_dtypes('object'))                                         # this got rid of the SK_ID_CURR column ...
bureau_cat['SK_ID_CURR'] = bureau['SK_ID_CURR']                                                     # so we have to replace it
bureau_cat = bureau_cat.groupby(by = ['SK_ID_CURR']).mean().reset_index()                           # tried sum - didn't change anything
print(bureau_cat.shape, "- shape of categorical bureau features (incl index)")                      # should be 132,250 x 23

# count feature for bureau
bureau_count = bureau.groupby(by = ['SK_ID_CURR'])['BU_CREDIT_ACTIVE'].count().reset_index()
bureau_count.rename(columns={'BU_CREDIT_ACTIVE':'COUNT_of_BUREAU'})   

# we are keeping bureau_num, bureau_cat and bureau_count - all will be merged into training data
del bureau                   # no longer need this table - its contents were transformed into bureau_num, bureau_cat, bureau_count
del bb_status                # no longer need this table - already merged into bureau
del bb_status_12             # no longer need this table - already merged into bureau
gc.collect()

### previous_application.csv
we will create previous_num and previous_cat - these will later be merged into train and into test.

It takes 7-8 minutes to calculate the CALC_RATE feature for all 1.6 million unique SK_ID_PREVs. So I ran them once, saved the output to PR_calcs.pkl, and then retrieve that information as needed, instead of recalculating every time.

In [ ]:
# This is the original function that derives CALC_RATE, INTEREST_PAID and INT_PRINC.

# def calc_rate(row):
#    return np.rate(row['CNT_PAYMENT'], -row['AMT_ANNUITY'], row['AMT_CREDIT'], 0, guess = 0.05, maxiter = 10)
# previous['CALC_RATE'] = previous.apply(calc_rate, axis=1)
# previous['INTEREST_PAID'] = previous['AMT_ANNUITY'] * previous['CNT_PAYMENT'] - previous['AMT_CREDIT']
# previous['INT_PRINC'] = previous['INTEREST_PAID'] / previous['AMT_CREDIT']

APR = train = pd.read_pickle('../input/aprdata/PR_calcs.pkl')
APR.drop(['SK_ID_CURR'], axis = 1, inplace = True)
APR.head(5)

In [ ]:
%%time

previous = pd.read_csv(f'{MainDir}/previous_application.csv')
print(previous.shape)
previous = previous.merge(APR, on='SK_ID_PREV', how='left')                      # the interest rate info that we just loaded
print(previous.shape)

previous.drop(['SK_ID_PREV'], axis = 1, inplace = True)                          # don't need this, merging everything straight to train/test

previous.columns = ['PR_'+column if column !='SK_ID_CURR' else column for column in previous.columns]
previous['PR_DAYS_LAST_DUE'].replace({365243: np.nan}, inplace = True)
previous['PR_DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace = True)
previous['PR_DAYS_FIRST_DUE'].replace(365243, np.nan, inplace = True)
previous['PR_DAYS_TERMINATION'].replace({365243: np.nan}, inplace = True)
previous['PR_DAYS_FIRST_DRAWING'].replace({365243: np.nan}, inplace = True)
previous['PR_CApp_RATIO'] = previous['PR_AMT_CREDIT'] / previous['PR_AMT_APPLICATION'] 
previous['PR_AG_RATIO'] = previous['PR_AMT_APPLICATION'] / previous['PR_AMT_GOODS_PRICE'] 
previous['PR_CAnnRATIO'] = previous['PR_AMT_CREDIT'] / previous['PR_AMT_ANNUITY'] 
previous['PR_CG_RATIO'] = previous['PR_AMT_CREDIT'] / previous['PR_AMT_GOODS_PRICE'] 

# Create numeric features by grouping on SK_ID_CURR and finding group means
previous_num = previous.groupby(by=['SK_ID_CURR']).mean().reset_index()         # group the numeric features by SK_ID_CURR
print(previous_num.shape, "- shape of numeric features (incl index)")         

# Create categorical features by creating dummies and then taking group means
previous_cat = pd.get_dummies(previous.select_dtypes('object'))                 # this got rid of the SK_ID_CURR column ...
previous_cat['SK_ID_CURR'] = previous['SK_ID_CURR']                             # so we have to replace it
previous_cat = previous_cat.groupby(by = ['SK_ID_CURR']).mean().reset_index()   # could try sum as well.
print(previous_cat.shape, "- shape of categorical features (incl index)")     

# we can keep previous_num and previous_cat - these will be merged into training data
del previous                 # no longer need this table - its contents were transformed into previous_num, previous_cat
del APR                      # already merged into previous
gc.collect()

### installment_payments.csv
only contains numeric features. we will create inst_num, which will later be merged into train and test

In [ ]:
inst = pd.read_csv(f'{MainDir}/installments_payments.csv')
inst.drop(['SK_ID_PREV'], axis=1, inplace = True)
inst.columns = ['IP_'+column if column != 'SK_ID_CURR' else column for column in inst.columns]

inst['PAY_PERCENT'] = inst['IP_AMT_INSTALMENT'] / inst['IP_AMT_PAYMENT']
inst['PAY_DIFF'] = inst['IP_AMT_INSTALMENT'] - inst['IP_AMT_PAYMENT']
inst['DPD'] = inst['IP_DAYS_ENTRY_PAYMENT'] - inst['IP_DAYS_INSTALMENT']
inst['DPD'] = inst['DPD'].apply(lambda x: x if x>0 else 0)
inst['DBD'] = inst['IP_DAYS_INSTALMENT'] - inst['IP_DAYS_ENTRY_PAYMENT']
inst['DBD'] = inst['DBD'].apply(lambda x: x if x>0 else 0)

inst_num = inst.groupby(by=['SK_ID_CURR']).mean().reset_index()         # group the numeric features by SK_ID_CURR
print(inst_num.shape, "- shape of numeric features (incl index)")       # should be 339,587 x 7

# we will keep inst_num and get rid of inst
del inst                    # don't need this anymore
gc.collect()

#### POS_CASH_balance.csv
we will create pos_num and pos_cat - these will later be merged into train and into test

In [ ]:
pos = pd.read_csv(f'{MainDir}/POS_CASH_balance.csv')
pos.drop(['SK_ID_PREV'], axis=1, inplace = True)
pos.columns = ['PO_'+column if column != 'SK_ID_CURR' else column for column in pos.columns]

pos_num = pos.groupby(by=['SK_ID_CURR']).mean().reset_index()            # group the numeric features by SK_ID_CURR
print(pos_num.shape, "- shape of numeric features (incl index)")         # should be 337,252 x 6

pos_cat = pd.get_dummies(pos.select_dtypes('object'))                    # this got rid of the SK_ID_CURR column ...
pos_cat['SK_ID_CURR'] = pos['SK_ID_CURR']                                # so we have to replace it
pos_cat = pos_cat.groupby(by = ['SK_ID_CURR']).mean().reset_index()      # could try sum as well.
print(pos_cat.shape, "- shape of categorical features (incl index)")     # should be 337,252 x 10

# we will keep pos_num and pos_cat
del pos
gc.collect()

### credit_card_balance.csv

In [ ]:
ccb = pd.read_csv(f'{MainDir}/credit_card_balance.csv')
ccb.drop(['SK_ID_PREV'], axis=1, inplace = True)
ccb.columns = ['CC_'+column if column != 'SK_ID_CURR' else column for column in ccb.columns]

ccb['DRAW_RATIO'] = ccb['CC_AMT_DRAWINGS_CURRENT'] / ccb['CC_CNT_DRAWINGS_CURRENT']
ccb['RECEIVE_RATIO'] = ccb['CC_AMT_RECIVABLE'] / ccb['CC_AMT_RECEIVABLE_PRINCIPAL']
ccb['RECEIVE_PER'] = ccb['CC_AMT_RECIVABLE'] / ccb['CC_AMT_TOTAL_RECEIVABLE']

ccb_num = ccb.groupby(by=['SK_ID_CURR']).mean().reset_index()            # group the numeric features by SK_ID_CURR
print(ccb_num.shape, "- shape of numeric features (incl index)")         # should be 103,558 x 24

ccb_cat = pd.get_dummies(ccb.select_dtypes('object'))                    # this got rid of the SK_ID_CURR column ...
ccb_cat['SK_ID_CURR'] = ccb['SK_ID_CURR']                                # so we have to replace it
ccb_cat = ccb_cat.groupby(by = ['SK_ID_CURR']).mean().reset_index()      # could try sum as well.
print(ccb_cat.shape, "- shape of categorical features (incl index)")     # should be 103,558 x 8

# we will keep ccb_num and ccb_cat
del ccb
gc.collect()

### Treatment of main data set (train or test)

In [ ]:
main = pd.read_csv(f'{MainDir}/{DFs[Switch]}')                          # either application_train or application_test depending on switch

# column drops - Megan suggested many others
main.drop(['ORGANIZATION_TYPE'], axis = 1, inplace = True)              # creates 58 dummies that don't do jack shit - let's get rid of this

# clean up this mess
main['FLAG_365243'] = main['DAYS_EMPLOYED'] == 365243                   # these people are mostly pensioners
main['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)         # DAYS_EMPLOYED is a powerful feature when you clean up the junk

# ratio features
main['CI_ratio'] = main['AMT_CREDIT'] / main['AMT_INCOME_TOTAL']        # credit-to-income ratio
main['AI_ratio'] = main['AMT_ANNUITY'] / main['AMT_INCOME_TOTAL']       # annuity-to-income ratio
main['AC_ratio'] = main['AMT_CREDIT'] / main['AMT_ANNUITY']             # credit to annuity - basically the term of the loan in years
main['CG_ratio'] = main['AMT_CREDIT'] / main['AMT_GOODS_PRICE']         # credit to goods price ratio - how much was financed?

# log features
main['log_INCOME'] = np.log(main['AMT_INCOME_TOTAL'])                    # log of income
main['log_ANNUITY'] = np.log(main['AMT_ANNUITY'])                        # log of annuity
main['log_CREDIT'] = np.log(main['AMT_CREDIT'])                          # log of credit
main['log_GOODS'] = np.log(main['AMT_GOODS_PRICE'])                      # log of goods price

# flag features
main['FLAG_CG_ratio'] = main['AMT_CREDIT'] > main['AMT_GOODS_PRICE']     # FLAG if you borrowed more than the price of the item
main['DAYS_ID_4200'] = main['DAYS_ID_PUBLISH'] < -4200                   # IDs more than about 14 years old are from USSR

# cleanup the ext_sources - and remember, these are columns 40:42 in the training data but 39:41 in the test data!
# objective of this section is to replace missing scores with the ROW average for the scores we do have, and not impute any scores.
# if you don't have ANY scores, you get a score of 0.2 - but there are only about a dozen of these
main['AVG_EXT'] = main.iloc[:,(40-Switch):(43-Switch)].sum(axis=1)/(3- main.iloc[:,(40-Switch):(43-Switch)].isnull().sum(axis=1))
main['AVG_EXT'].replace(np.nan, 0.2, inplace = True)       
main.EXT_SOURCE_1.fillna(main.AVG_EXT, inplace=True)
main.EXT_SOURCE_2.fillna(main.AVG_EXT, inplace=True)
main.EXT_SOURCE_3.fillna(main.AVG_EXT, inplace=True)
main['EXT_SOURCE_MAX'] = np.max(main.iloc[:,(40-Switch):(43-Switch)], axis = 1)
main['EXT_SOURCE_MIN'] = np.min(main.iloc[:,(40-Switch):(43-Switch)], axis = 1)

In [ ]:
# merge features
DFs_to_merge = [bureau_cat, bureau_count, bureau_num, ccb_cat, ccb_num,
 inst_num, pos_cat, pos_num, previous_cat, previous_num]

for i in range (0, len(DFs_to_merge)):
    main = main.merge(DFs_to_merge[i], on='SK_ID_CURR', how='left')

print(main.shape, "- shape of data after all merges") 

# get rid of any +/- infinity we might have missed
main.replace([np.inf, -np.inf], 0, inplace = True)

del bureau_cat
del bureau_count
del bureau_num
del ccb_cat
del ccb_num
del inst_num
del pos_cat
del pos_num
del previous_cat
del previous_num
gc.collect()

### Export our data set to pickle

In [ ]:
main.to_pickle(f'./{FNs[Switch]}', compression='infer', storage_options=None)